In [13]:
from mwrogue.esports_client import EsportsClient
import pandas as pd
import os
import time

In [14]:
site = EsportsClient("lol")
response = site.cargo_client.query(
    tables="RegionStatuses=R",
    fields="R.Region, R.Year, R.Status",
    where="R.Year  >= '2020' AND (R.Status = 'Play-In' OR R.Status = 'Autoqualify')",
)
df_regioes = pd.DataFrame(response)
df_regioes = df_regioes.sort_values(by='Year', ascending=False)
df_regioes = df_regioes.drop_duplicates(subset='Region', keep='first')
df_regioes.insert(0, 'Region_id', range(1, len(df_regioes) + 1))

df_regioes.to_csv('regioes.csv', index=False)

In [15]:
response = site.cargo_client.query(
    tables="Leagues=L,RegionStatuses=R",
    join_on="L.Region=R.Region",
    fields="L.League, L.League_Short, L.Region, R.Year, R.Status",
    where="R.Year  >= '2020' AND (R.Status = 'Play-In' OR R.Status = 'Autoqualify') AND L.Level = 'Primary' AND L.IsOfficial = 'Yes'",
)
df_ligas = pd.DataFrame(response)
df_ligas = df_ligas.sort_values(by='Year', ascending=False)
df_ligas = df_ligas.drop_duplicates(subset='League', keep='first')
df_ligas.insert(0, 'League_id', range(1, len(df_ligas) + 1))
df_ligas.insert(1, 'Region_id', df_ligas['Region'].map(df_regioes.set_index('Region')['Region_id']))
df_ligas = df_ligas.drop('Region', axis=1)

df_ligas.to_csv('ligas.csv', index=False)

In [16]:
from datetime import datetime
response = site.cargo_client.query(
    tables="Tournaments=T, Leagues=L",
    join_on="T.League=L.League",
    fields="T.Name, T.DateStart , T.League, T.Region, T.Year,T.TournamentLevel,T.IsPlayoffs",
    where="T.TournamentLevel  = 'Primary' AND T.Year>='2024' AND L.Level = 'Primary' AND L.IsOfficial = 'Yes'",
)
df_torneios = pd.DataFrame(response)
df_torneios.insert(0, 'Tournament_id', range(1, len(df_torneios) + 1))
df_torneios.insert(1, 'Region_id', df_torneios['Region'].map(df_regioes.set_index('Region')['Region_id']))
df_torneios.insert(2, 'League_id', df_torneios['League'].map(df_ligas.set_index('League')['League_id']))
df_torneios = df_torneios.drop(columns=['Region','League'], axis=1)
df_torneios = df_torneios.dropna(subset=['Region_id','League_id'])
df_torneios[['Region_id','League_id']] = df_torneios[['Region_id','League_id']].astype(int)
df_torneios['InProgress'] = [0 if pd.isna(data) or datetime.strptime(data, '%Y-%m-%d') > datetime.today() else 1 for data in df_torneios['DateStart']]
df_torneios['LastGame'] = ''
df_torneios.to_csv('torneios.csv', index=False)

In [17]:
torneios = pd.read_csv('torneios.csv')
where_tournament = ''
for i, torneio in enumerate(torneios['Name']):
    if i == len(torneios['Name'])-1: where_tournament += f"T.Name = '{torneio}'"
    else: where_tournament += f"T.Name = '{torneio}' OR "

In [18]:
response = site.cargo_client.query(
    tables="Champions=Ch",
    fields="Ch.Name"
)
for i,campeao in enumerate(response):
    if campeao['Name'] == 'Nunu &amp; Willump':
        response[i] = dict([('Name', 'Nunu & Willump')])
df_campeoes = pd.DataFrame(response)
df_campeoes.insert(0, 'Champion_id', range(1, len(df_campeoes) + 1))
df_campeoes['CaminhoImagem'] = df_campeoes['Name'].apply(lambda x: os.path.join(r"\png_campeoes", f"{x}.png"))
df_campeoes.to_csv('campeoes.csv', index=False)

In [19]:
# url = "https://lol.fandom.com/wiki/Category:Champion_Square_Images?filefrom=YuumiSquare.png#mw-category-media"

# # Configuração do Selenium para abrir o navegador no seu computador
# driver = webdriver.Chrome()

# driver.get(url)
# time.sleep(3)  # Espera um pouco para garantir que a página foi totalmente carregada

# # Rola a página para baixo para carregar mais itens
# body = driver.find_element(by=By.TAG_NAME, value="body")
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(3)  # Aguarda novamente após rolar a página

# soup = BeautifulSoup(driver.page_source, 'html.parser')
# image_tags = soup.find_all('img')

# for img_tag in image_tags:
#     data_image_key = img_tag.get('data-image-key', '')
#     champion_name = data_image_key.replace('Square.png', '')

#     if champion_name in df_campeoes['Name'].values:
#         if os.path.exists(os.path.join("png_campeoes", f"{champion_name}.png")):
#             continue
#         print(champion_name)
#         img_url = urljoin(url, img_tag['src'])
#         print(img_url)

#         # Espera até que o campeão esteja na URL da imagem
#         while champion_name not in img_url:
#             time.sleep(3)

#         # Baixa a imagem usando requests
#         img_data = requests.get(img_url).content

#         # Salva a imagem
#         with open(os.path.join("png_campeoes", f"{champion_name}.png"), 'wb') as f:
#             f.write(img_data)

# # Fecha o navegador após a conclusão
# driver.quit()

In [20]:
response = site.cargo_client.query(
    tables="ScoreboardGames=SG, Tournaments=T",
    join_on="SG.OverviewPage=T.OverviewPage",
    fields="SG.GameId, SG.Tournament, SG.DateTime_UTC, SG.Team1, SG.Team2, SG.WinTeam, SG.Gamelength_Number, SG.Patch, SG.Team1Bans, SG.Team2Bans, SG.Team1Picks, SG.Team2Picks, SG.Team1Dragons, SG.Team2Dragons, SG.Team1Barons, SG.Team2Barons, SG.Team1Towers, SG.Team2Towers, SG.Team1Gold, SG.Team2Gold, SG.Team1Kills, SG.Team2Kills, SG.Team1RiftHeralds, SG.Team2RiftHeralds, SG.Team1Inhibitors, SG.Team2Inhibitors",
    where=f"{where_tournament}",
    order_by="SG.DateTime_UTC"
)
df_partidas = pd.DataFrame(response)
df_partidas.insert(1, 'Tournament_id', df_partidas['Tournament'].map(df_torneios.set_index('Name')['Tournament_id']))
df_partidas = df_partidas.drop('Tournament', axis=1)
df_partidas.to_csv('partidas.csv', index=False)

for torneio in df_torneios['Tournament_id']:
    if df_torneios.loc[df_torneios['Tournament_id'] == torneio, 'InProgress'].values[0] == 1:
        lastgamedate = max(df_partidas.loc[(df_partidas['Tournament_id'] == torneio), 'DateTime UTC'], default=None)
        if lastgamedate and (datetime.today() - datetime.strptime(lastgamedate, '%Y-%m-%d %H:%M:%S')).days > 7:
            df_torneios.loc[df_torneios['Tournament_id'] == torneio, 'InProgress'] = 0
        lastgameid = df_partidas.loc[(df_partidas['DateTime UTC'] == lastgamedate),'GameId']
        df_torneios.loc[df_torneios['Tournament_id'] == torneio, 'LastGame'] = lastgameid.values
df_torneios.to_csv('torneios.csv', index=False)


In [21]:
df_partidas = pd.read_csv('partidas.csv')
team_names = pd.unique(df_partidas[['Team1', 'Team2']].values.ravel('K'))

df_times = pd.DataFrame(columns=['Name', 'Tournament_id', 'Towers_avg', 'Dragons_avg', 'Barons_avg', 'Heralds_avg',
                                  'Inhib_avg', 'Kills_avg', 'Gold/min', 'Win_rate', 'Gamelength_win_avg'])

for team_name in team_names:
    team1_rows = df_partidas[df_partidas['Team1'] == team_name]

    team2_rows = df_partidas[df_partidas['Team2'] == team_name]

    team_rows = pd.concat([team1_rows, team2_rows])

    # Cálculo das médias
    towers_avg = (team1_rows['Team1Towers'].astype(int).sum()+team2_rows['Team2Towers'].astype(int).sum())/len(team_rows)
    dragons_avg = (team1_rows['Team1Dragons'].astype(int).sum()+team2_rows['Team2Dragons'].astype(int).sum())/len(team_rows)
    barons_avg = (team1_rows['Team1Barons'].astype(int).sum()+team2_rows['Team2Barons'].astype(int).sum())/len(team_rows)
    heralds_avg = (team1_rows['Team1RiftHeralds'].sum()+team2_rows['Team2RiftHeralds'].sum())/len(team_rows)
    inhib_avg = (team1_rows['Team1Inhibitors'].astype(int).sum()+team2_rows['Team2Inhibitors'].astype(int).sum())/len(team_rows)
    kills_avg = (team1_rows['Team1Kills'].astype(int).sum()+team2_rows['Team2Kills'].astype(int).sum())/len(team_rows)
    gold_per_min = (team1_rows['Team1Gold'].astype(int).sum()+team2_rows['Team2Gold'].astype(int).sum())/(team_rows['Gamelength Number'].astype(float).sum())
    win_rate = team_rows['WinTeam'].eq(team_name).mean()
    team_rows['Gamelength Number'] = team_rows['Gamelength Number'].astype(float)
    gamelength_win_avg = team_rows[team_rows['WinTeam'] == team_name]['Gamelength Number'].mean()


    team_data = pd.DataFrame({
        'Name': [team_name],
        'Tournament_id': [team_rows['Tournament_id'].iloc[0]],
        'Towers_avg': [towers_avg],
        'Dragons_avg': [dragons_avg],
        'Barons_avg': [barons_avg],
        'Heralds_avg': [heralds_avg],
        'Inhib_avg': [inhib_avg],
        'Kills_avg': [kills_avg],
        'Gold/min': [gold_per_min],
        'Win_rate': [win_rate],
        'Gamelength_win_avg': [gamelength_win_avg]
    })

    # Adição do DataFrame ao DataFrame df_times
    df_times = pd.concat([df_times, team_data], ignore_index=True)

# Salvando o DataFrame df_times como um arquivo CSV
df_times.insert(0, 'Team_id', range(1, len(df_times) + 1))
df_times.to_csv('times.csv', index=False)

C:\Users\Bianchi\AppData\Local\Temp\ipykernel_4072\2753148592.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_times = pd.concat([df_times, team_data], ignore_index=True)


In [22]:
response = site.cargo_client.query(
    tables="ScoreboardGames=SG, ScoreboardPlayers=SP, Tournaments=T",
    join_on="SG.GameId=SP.GameId, SG.OverviewPage=T.OverviewPage",
    fields="SP.Name, SP.Team, SG.Tournament, SP.Role, SP.GameId, SP.PlayerWin, SP.Champion, SG.Gamelength_Number, SP.Kills, SP.Deaths, SP.Assists, SP.Gold, SP.CS, SP.DamageToChampions, SP.VisionScore",
    where=f"{where_tournament}"
)
df_player_stats = pd.DataFrame(response)
df_player_stats.to_csv('performance_jogadores.csv', index=False)

In [23]:
df_player_stats = pd.read_csv('performance_jogadores.csv')
player_names = df_player_stats['Name'].unique()

df_players = pd.DataFrame(columns=['Name', 'Team', 'Role', 'Kills_avg', 'Assists_avg', 'Deaths_avg',
                                     'CS/min', 'Gold/min', 'Damage/min', 'Vision/min'])
for player in player_names:
    player_rows = df_player_stats[df_player_stats['Name'] == player]

    # Cálculo das médias
    kills_avg = player_rows['Kills'].mean()
    assists_avg = player_rows['Assists'].mean()
    deaths_avg = player_rows['Deaths'].mean()
    cs_min = player_rows['CS'].sum()/(player_rows['Gamelength Number'].sum())
    gold_min = player_rows['Gold'].sum()/(player_rows['Gamelength Number'].sum())
    damage_min = player_rows['DamageToChampions'].sum()/(player_rows['Gamelength Number'].sum())
    vision_min = player_rows['VisionScore'].sum()/(player_rows['Gamelength Number'].sum())


    player_data = pd.DataFrame({
        'Name': [player],
        'Team': [player_rows['Team'].iloc[0]],
        'Role': [player_rows['Role'].iloc[0]],
        'Kills_avg': [kills_avg],
        'Assists_avg': [assists_avg],
        'Deaths_avg': [deaths_avg],
        'CS/min': [cs_min],
        'Gold/min': [gold_min],
        'Damage/min': [damage_min],
        'Vision/min': [vision_min]
    })

    # Adição do DataFrame ao DataFrame times_df
    df_players = pd.concat([df_players, player_data], ignore_index=True)

# Salvando o DataFrame times_df como um arquivo CSV
df_players.insert(0, 'Player_id', range(1, len(df_players) + 1))
df_players.insert(1, 'Team_id', df_players['Team'].map(df_times.set_index('Name')['Team_id']))
df_players = df_players.drop('Team', axis=1)

df_player_stats.insert(0, 'Player_id', df_player_stats['Name'].map(df_players.set_index('Name')['Player_id']))
df_player_stats.insert(1, 'Team_id', df_player_stats['Team'].map(df_times.set_index('Name')['Team_id']))
df_player_stats.insert(2, 'Tournament_id', df_player_stats['Tournament'].map(df_torneios.set_index('Name')['Tournament_id']))
df_player_stats.insert(3, 'Champion_id', df_player_stats['Champion'].map(df_campeoes.set_index('Name')['Champion_id']))
df_player_stats = df_player_stats.drop(columns=['Name','Team','Tournament','Champion'], axis=1)



df_players.to_csv('jogadores.csv', index=False)
df_player_stats.to_csv('performance_jogadores.csv', index=False)

C:\Users\Bianchi\AppData\Local\Temp\ipykernel_4072\670136991.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_players = pd.concat([df_players, player_data], ignore_index=True)


In [24]:
# regioes = pd.read_csv('regioes.csv')
# where_regions = ''
# for i, regiao in enumerate(regioes['Region']):
#     if i == len(regioes['Region'])-1: where_regions += f"T.Region = '{regiao}'"
#     else: where_regions += f"T.Region = '{regiao}' OR "
# response = site.cargo_client.query(
#     tables="Teams=T, RegionStatuses=R",
#     join_on="T.Region=R.Region",
#     fields="T.Name, T.Short, T.Region, T.Image",
#     where=f"({where_regions}) AND T.Name != 'None' AND T.IsDisbanded = 'No' AND (R.Year = '2023' OR ((R.Region = 'Europe' OR R.Region = 'Turkey' OR R.Region = 'Commonwealth of Independent States') AND R.Year = '2022' ))"
# )
# response